In [ ]:
import torch
import pandas as pd 
from ultralytics import YOLO
import numpy as np
import supervision as sv
import logging
from tkinter import Tk,filedialog
import os 
from pathlib import Path
import csv
import cv2
from typing import Optional
import warnings

In [ ]:
model = YOLO('../Models/2025_v0.pt')
#source_file =  add code for file diaglouge to open to laod video 
#target_file = get the file name freom sourcepath and add_processed before the file extgension. 
logging.basicConfig(level=logging.INFO)
logging.info('Started Inference')

root=Tk()
root.withdraw()
source_file = filedialog.askopenfilename(title="Select source video file",filetypes=[("Video files", "*.mp4;*.avi;*.mov"), ("All files", "*.*")])
if not source_file:
    logging.error("No source file selected. Exiting.")
    exit()
directory,filename = os.path.split(source_file)
name,ext = os.path.splitext(filename)
target_file_name = name + '_processed' + ext
target_file = os.path.join(directory,target_file_name)

In [ ]:
def run_inference(
    model: YOLO,
    frame: np.ndarray,
    frame_number: int,
    *,
    tracker,                 # e.g. tracker = sv.ByteTrack()
    state: dict,             # mutable dict, must contain or store state["ball_id"]
    save_csv: bool = False,
    writer: Optional = None,
    conf_threshold: float = 0.25,
    iou_threshold: float = 0.45
) -> np.ndarray:
    """
    Runs YOLO → ByteTrack → annotate → optionally log to CSV.

    Args:
      model          : your YOLO instance
      frame          : HxWx3 video frame array
      frame_number   : frame index

    Keyword Args:
      tracker        : a sv.ByteTrack() (or TrackerDeepSort) instance
      state          : dict to hold {'ball_id': Optional[int]}
      save_csv       : if True, must also pass `writer`
      writer         : csv.writer (required if save_csv=True)
      conf_threshold : YOLO confidence cutoff
      iou_threshold  : YOLO NMS IoU threshold
    Returns:
      Annotated frame
    """
    # --- sanity checks ---
    if save_csv and writer is None:
        raise ValueError("`writer` is required when save_csv=True")
    if not save_csv and writer is not None:
        raise ValueError("`writer` only allowed when save_csv=True")

    # 1) YOLO detect + built-in NMS
    yolo_res = model(frame, conf=conf_threshold, iou=iou_threshold)[0]
    dets     = sv.Detections.from_ultralytics(yolo_res)

    # 2) ByteTrack association
    tracked = tracker.update_with_detections(dets)

    # 3) On first valid frame, pick which track is the ball
    if state.get("ball_id") is None and tracked.tracker_id.size>0:
        best_idx         = int(np.argmax(tracked.confidence))
        state["ball_id"] = tracked.tracker_id[best_idx]
    print('Getting ball id')
    ball_id = state.get("ball_id")

    # 4) Filter out everything except our chosen ball
    if ball_id is not None:
        # build mask of where tracked.tracker_id == ball_id
        mask = [tid == ball_id for tid in tracked.tracker_id]
        if any(mask):
            # keep only those entries
            keep_idxs = [i for i, m in enumerate(mask) if m]
            xyxy      = tracked.xyxy[keep_idxs]
            confs     = tracked.confidence[keep_idxs]
            tids      = [tracked.tracker_id[i] for i in keep_idxs]
        else:
            # lost track this frame → fall back to raw detections
            xyxy  = dets.xyxy
            confs = dets.confidence
            tids  = [None] * len(dets.xyxy)
    else:
        # before lock-in, log all raw detections
        xyxy  = dets.xyxy
        confs = dets.confidence
        tids  = [None] * len(dets.xyxy)

    # rebuild a Detections object for annotation
    dets_to_draw = sv.Detections(
        xyxy       = np.array(xyxy),
        confidence = np.array(confs),
        class_id   = np.zeros(len(xyxy), dtype=int)
    )

    # 5) Annotate bounding boxes
    img = sv.BoxAnnotator().annotate(frame, dets_to_draw)

    # 6) Overlay frame number
    cv2.putText(
        img, str(frame_number),
        org=(10, 30),
        fontFace=cv2.FONT_HERSHEY_COMPLEX,
        fontScale=1,
        color=(255, 0, 0),
        thickness=2,
        lineType=cv2.LINE_AA
    )

    # 7) Optionally log to CSV
    if save_csv:
        # write one row per box we drew
        for det_idx, (xy, tid,conf) in enumerate(zip(dets_to_draw.xyxy, tids,dets_to_draw.confidence), start=1):
            x1, y1, x2, y2 = map(float, xy)
            c_x = (x1 + x2) / 2
            c_y = (y1 + y2) / 2
            writer.writerow([
                frame_number,
                tid,      # tracker_id (or None if not yet locked)
                det_idx,  # per-frame detection index
                conf,
                c_x, c_y,
                x1, y1, x2, y2
            ])

    return img

In [ ]:
try:
    csv_file = '../Data/detections.csv'
    f = open(csv_file,mode='w',newline='')
    writer = csv.writer(f)
    writer.writerow(['frame_number','tracker_id','detection_id','confidence','c_x','c_y','x1','x2','y1','y2'])
    tracker = sv.ByteTrack()
    state   = {"ball_id": None}
    warnings.filterwarnings("error", message="The truth value of an empty array is ambiguous")
    sv.process_video(source_path=source_file,
                     target_path=target_file,
                     callback=lambda frame,frame_number:run_inference(model, frame,frame_number,tracker=tracker,state=state,save_csv=True,writer=writer))
    logging.info('Inference completed')
finally:
    f.close()
    
